## We sort the Walkway data by zipcode and census tract

In [1]:
import pandas as pd    
import geopandas as gpd
import datetime
from geopandas.tools import geocode
from shapely.geometry import Point
import numpy as np
import shapely
from shapely import speedups
speedups.enabled
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})

### Match and join zipcodes and tracts columns

In [2]:
# Loads zip code boundary data
zipcodes_url = 'https://opendata.arcgis.com/datasets/83fc2e72903343aabff6de8cb445b81c_2.geojson'
zipcodes_file = gpd.read_file(zipcodes_url)

In [7]:
# Creates a dataframe "zips" of zip codes, and areas, and geometries associated to the zipcodes
zip_cols = ['ZIPCODE', 'SHAPE_Area', 'geometry']
zips = zipcodes_file.loc[:,zip_cols]
zips['ZIPCODE'] = zips['ZIPCODE'].astype(int)
zips.sort_values(by='ZIPCODE', inplace=True)
zips.set_index(np.arange(0,len(zips),1), inplace=True)
zips.index.name = 'ID'

In [8]:
zips.head()

,ZIPCODE,SHAPE_Area,geometry
ID,,,
0,98001,5.324255e+08,"POLYGON ((-122.22992 47.35377, -122.22992 47.3..."
1,98002,2.055688e+08,"POLYGON ((-122.22921 47.35375, -122.22921 47.3..."
2,98003,3.170769e+08,"POLYGON ((-122.30960 47.35795, -122.31323 47.3..."
3,98004,2.014501e+08,"MULTIPOLYGON (((-122.21458 47.65298, -122.2147..."
4,98005,2.113641e+08,"POLYGON ((-122.15876 47.66065, -122.16408 47.6..."


In [9]:
# Loads census tract boundary data
census_url = 'https://opendata.arcgis.com/datasets/de58dc3e1efc49b782ab357e044ea20c_9.geojson'
census_file = gpd.read_file(census_url)

In [16]:
# Create a dataframe "tracts" of tract no., and areas and geometries associated with each tract no.
# 'NAME10' is the tract no.
tract_cols = ['NAME10', 'SHAPE_Area', 'geometry']
tracts = census_file.loc[:,tract_cols]
tracts['NAME10'] = tracts['NAME10'].astype(float)
tracts.sort_values(by='NAME10', inplace=True)
tracts.set_index(np.arange(0,len(tracts),1), inplace=True)
tracts.index.name = 'ID'

# Filters out tracts with irregular traffic data
filter1 = tracts.NAME10 == 260.01
filter2 = tracts.NAME10 == 264.0
indices1 = tracts[filter1].index
indices2 = tracts[filter2].index
tracts.drop(indices1, inplace=True)
tracts.drop(indices2, inplace=True)

In [17]:
tracts.head()

,NAME10,SHAPE_Area,geometry
ID,,,
0,1.00,1.990374e+07,"POLYGON ((-122.29605 47.73378, -122.29651 47.7..."
1,2.00,3.537107e+07,"POLYGON ((-122.29634 47.72109, -122.29636 47.7..."
2,3.00,1.302888e+07,"POLYGON ((-122.32555 47.73407, -122.32561 47.7..."
3,4.01,1.467846e+07,"POLYGON ((-122.34501 47.72633, -122.34502 47.7..."
4,4.02,1.936217e+07,"POLYGON ((-122.34498 47.72324, -122.34633 47.7..."


In [22]:
# Joins zip codes with census tracts
ziptracts = gpd.sjoin(zips, tracts, op='intersects')
zip_cols = ['ZIPCODE', 'NAME10', 'SHAPE_Area_left', 'geometry']
ziptracts = ziptracts[zip_cols]

In [23]:
# Defines x and y boundaries of area covered by tracts
minx = min(tracts.geometry.bounds.minx)
maxx = max(tracts.geometry.bounds.maxx)
miny = min(tracts.geometry.bounds.miny)
maxy = max(tracts.geometry.bounds.maxy)

# Shrinks the area covered by ziptracts to the area covered by tracts
minx_filter = minx < ziptracts.geometry.bounds.minx
maxx_filter = maxx > ziptracts.geometry.bounds.maxx
miny_filter = miny < ziptracts.geometry.bounds.miny
maxy_filter = maxy > ziptracts.geometry.bounds.maxy

ziptracts = ziptracts[minx_filter & maxx_filter & miny_filter & maxy_filter]

### Sort Walkway data by zipcodes and tracts columns

In [26]:
# walkway dataframe, note that it contains bike lanes data
df_walkway = gpd.read_file('Street_Network_Database_SND.geojson')

In [48]:
walk_columns = ['SEGMENT_TY', 'SNDSEG_UPD', 'SHAPE_Leng', 'geometry']
# 'SEGMENT_TY' -- segment type, walkway data have type 5
# 'SNDSEG_UPD' -- date
# 'SHAPE_Leng' -- length of walkway
walkdat = df_walkway[walk_columns]
walkdat.head()

,SEGMENT_TY,SNDSEG_UPD,SHAPE_Leng,geometry
0,1,2007-04-22,291.426335,"LINESTRING (-122.32286 47.52981, -122.32287 47..."
1,1,2007-04-22,431.467882,"LINESTRING (-122.32400 47.61848, -122.32401 47..."
2,1,2004-05-19,162.446146,"LINESTRING (-122.29191 47.76490, -122.29126 47..."
3,6,2004-05-19,79.522621,"LINESTRING (-122.30780 47.61410, -122.30748 47..."
4,1,2012-10-12,643.835648,"LINESTRING (-122.38213 47.67455, -122.38474 47..."


In [50]:
# format date as year only
walkdat_years = []
for year in walkdat['SNDSEG_UPD'].values:
    walkdat_years.append(int(year[0:4]))
    
walkdat['YEAR'] = pd.Series(walkdat_years)

In [51]:
# extract walkways from dataframe, denoted by SEGMENT TYPE of 5
walkdat_clean = walkdat[walkdat['SEGMENT_TY']==5]
walkdat_clean = walkdat_clean.drop(columns=['SEGMENT_TY'])
walkdat_clean = walkdat_clean.dissolve('YEAR')
walkdat_clean.head()

,geometry,SNDSEG_UPD,SHAPE_Leng
YEAR,,,
2004,"MULTILINESTRING ((-122.31491 47.69172, -122.31...",2004-07-19,165.967550
2005,"LINESTRING (-122.32653 47.64449, -122.32653 47...",2005-05-04,412.348264
2008,"LINESTRING (-122.38645 47.54191, -122.38646 47...",2008-04-09,465.983637
2014,"LINESTRING (-122.38163 47.69059, -122.38162 47...",2014-12-18,665.738612
2015,"MULTILINESTRING ((-122.34646 47.53375, -122.34...",2015-03-27,666.636653


In [61]:
# oraganize walks by zipcodes and tracts
walkdat_by_ziptract = gpd.sjoin(ziptracts, walkdat_clean, op='intersects')
walkdat_by_ziptract.sort_values(by='ZIPCODE', inplace=True)
# we dissolve the data with 'ZIPCODE' and 'YEAR', here 'YEAR' is as 'index_right',
walkdat_by_ziptract.dissolve(by=['index_right','ZIPCODE'])
# rename 'index right' as 'YEAR'
walkdat_by_ziptract.rename(columns={'index_right': 'YEAR'}, inplace=True)
walkdat_by_ziptract.drop(columns=['geometry','SNDSEG_UPD'])

,ZIPCODE,NAME10,SHAPE_Area_left,YEAR,SHAPE_Leng
ID,,,,,
58,98101,81.00,1.470012e+07,2019,306.056518
58,98101,82.00,1.470012e+07,2019,306.056518
58,98101,83.00,1.470012e+07,2019,306.056518
58,98101,80.02,1.470012e+07,2019,306.056518
58,98101,84.00,1.470012e+07,2019,306.056518
...,...,...,...,...,...
81,98144,91.00,9.241941e+07,2020,354.861458
81,98144,86.00,9.241941e+07,2020,354.861458
81,98144,101.00,9.241941e+07,2020,354.861458
